### Word Group In Youtube Firebase

In [1]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [2]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [3]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [4]:
# language pair
lang_folder = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# language detail num
if lang_folder == "English":
    lang_id = "1"
elif lang_folder == "Turkish":
    lang_id = "2"
elif lang_folder == "French":
    lang_id = "3"
elif lang_folder == "Spanish":
    lang_id = "4"
elif lang_folder == "Portuguese":
    lang_id = "5"
elif lang_folder == "Dutch":
    lang_id = "6"
elif lang_folder == "Italian":
    lang_id = "7"
elif lang_folder == "German":
    lang_id = "8"
elif lang_folder == "Arabic":
    lang_id = "9"
else:
    pass

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"Language = {lang_folder}\nLang Id = {lang_id}\nFile Extention = {file_ext}")

Language = French
Lang Id = 3
File Extention = 6


In [5]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Result/2-Word Group In Youtube Sentence Firebase/{lang_folder.capitalize()}"

#Path(path).mkdir(parents=True, exist_ok=True)

In [6]:
#cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Language Level/Api Key/language-levels-firebase-adminsdk-u1m3n-87de34db73.json")  # Language Level
app = firebase_admin.initialize_app(cred)

In [7]:
fire_db = firestore.client()

#### Read Data

In [8]:
parent_folder_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Language Level/Data/1-Word Group In Youtube Sentence"

In [9]:
df_28 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_28_Word{file_ext}.csv")
df_28 = df_28.head(200)
df_28

,search_string,start_time,end_time,sentence,video_id,video_url
0,cest pour ça que je vous le,672,675,cest pour ça que je vous le montre,aDBLqp0bIPw,https://www.youtube.com/watch?v=aDBLqp0bIPw&t=...
1,de de ce de et cest un des,2299,2303,de de ce de et cest un des,_vwTpcrrNGI,https://www.youtube.com/watch?v=_vwTpcrrNGI&t=...
2,mais cest pas de ça que,2352,2354,temps mais cest pas de ça que,seWzxrNRuZc,https://www.youtube.com/watch?v=seWzxrNRuZc&t=...
3,et cest pour ça que jai,405,406,et cest pour ça que jai réussi,q74if9KPmt0,https://www.youtube.com/watch?v=q74if9KPmt0&t=...
4,que cest pour nous cest,3677,3682,que cest pour nous cest par,seWzxrNRuZc,https://www.youtube.com/watch?v=seWzxrNRuZc&t=...
...,...,...,...,...,...,...
195,et nous,145,148,et nous,vIelXxpoCf4,https://www.youtube.com/watch?v=vIelXxpoCf4&t=...
196,pour ça,366,367,pour ça,BKNQfWBFTHk,https://www.youtube.com/watch?v=BKNQfWBFTHk&t=...
197,et vous,46,49,et vous,LsU3Y_Y2J38,https://www.youtube.com/watch?v=LsU3Y_Y2J38&t=46s
198,pas des,459,463,pas des,8dZAfyvIjho,https://www.youtube.com/watch?v=8dZAfyvIjho&t=...


In [10]:
df_200 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_200_Word{file_ext}.csv")
df_200 = df_200.head(200)
df_200

,search_string,start_time,end_time,sentence,video_id,video_url
0,quand même quoi donc parce que voilà cest fait...,157,163,quand même quoi donc parce que voilà cest fait...,0_sv895JPlc,https://www.youtube.com/watch?v=0_sv895JPlc&t=...
1,tout je fais ça va prendre trop de temps et au...,3441,3445,tout je fais ça va prendre trop de temps et au...,jfaNVKkLMi4,https://www.youtube.com/watch?v=jfaNVKkLMi4&t=...
2,cest quelquun qui est comme moi mais qui nest ...,3260,3265,lautre cest quelquun qui est comme moi mais qu...,oMslm9BSPjs,https://www.youtube.com/watch?v=oMslm9BSPjs&t=...
3,là tu es vraiment avec toi même et ça fait vra...,1485,1492,au quotidien là tu es vraiment avec toi même e...,9r48eQQ7_E4,https://www.youtube.com/watch?v=9r48eQQ7_E4&t=...
4,de la vie mais voilà ça fait quand même mal et...,1235,1239,aléas de la vie mais voilà ça fait quand même ...,tqYpLBLVHU8,https://www.youtube.com/watch?v=tqYpLBLVHU8&t=...
...,...,...,...,...,...,...
195,alors avait dit alors quelque chose comme,743,746,et isaïe alors avait dit alors quelque chose c...,0xbw7lp4SFg,https://www.youtube.com/watch?v=0xbw7lp4SFg&t=...
196,ça fait trop de bien de prendre un peu le,950,964,ça fait trop de bien de prendre un peu le soleil,mB0EaUItdd0,https://www.youtube.com/watch?v=mB0EaUItdd0&t=...
197,jai passé un petit peu de temps juste ici,445,448,jai passé un petit peu de temps juste ici,rqx5byLTA_g,https://www.youtube.com/watch?v=rqx5byLTA_g&t=...
198,et en plus si vous allez toujours au même,1138,1140,et en plus si vous allez toujours au même endroit,963GImaM8ps,https://www.youtube.com/watch?v=963GImaM8ps&t=...


In [11]:
df_1000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_1000_Word{file_ext}.csv")
df_1000 = df_1000.head(200)
df_1000

,search_string,start_time,end_time,sentence,video_id,video_url
0,voilà oui on pourrait passer encore beaucoup p...,5913,5920,voilà oui on pourrait passer encore beaucoup p...,Tndgvn3FoZU,https://www.youtube.com/watch?v=Tndgvn3FoZU&t=...
1,jai fait un bon petit tour de la maison elle e...,262,269,jai fait un bon petit tour de la maison elle e...,4wq9PNG0uvA,https://www.youtube.com/watch?v=4wq9PNG0uvA&t=...
2,on serait pas là où ça veut dire que par contr...,3191,3196,néandertaliens on serait pas là où ça veut dir...,Tndgvn3FoZU,https://www.youtube.com/watch?v=Tndgvn3FoZU&t=...
3,alors là je lui ai vraiment parlé avec mon coe...,2237,2243,alors là je lui ai vraiment parlé avec mon coe...,EUCds0hVVHY,https://www.youtube.com/watch?v=EUCds0hVVHY&t=...
4,pour savoir comment ça allait oui cétait pas l...,99,104,pour savoir comment ça allait oui cétait pas l...,O3wgZ9GE33M,https://www.youtube.com/watch?v=O3wgZ9GE33M&t=99s
...,...,...,...,...,...,...
195,si jai bien compris en fait elle vient de là h...,647,657,si jai bien compris en fait elle vient de là h...,TwDddYiivds,https://www.youtube.com/watch?v=TwDddYiivds&t=...
196,de la sorte son dieu pour elle seul un dieu to...,901,911,de la sorte son dieu pour elle seul un dieu to...,jL3znanJJC4,https://www.youtube.com/watch?v=jL3znanJJC4&t=...
197,est bien daccord mais par contre il ne se pass...,104,109,est bien daccord mais par contre il ne se pass...,cj2J1x6md1k,https://www.youtube.com/watch?v=cj2J1x6md1k&t=...
198,ou soit la personne te dit je connais quelquun...,781,785,ou soit la personne te dit je connais quelquun...,GUQKIROlp2w,https://www.youtube.com/watch?v=GUQKIROlp2w&t=...


In [12]:
df_5000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_5000_Word{file_ext}.csv")
df_5000 = df_5000.head(200)
df_5000

,search_string,start_time,end_time,sentence,video_id,video_url
0,attend papa je vais te montrer des photos de m...,2124,2130,attend papa je vais te montrer des photos de m...,u41ujNodvnM,https://www.youtube.com/watch?v=u41ujNodvnM&t=...
1,mais pas nimporte quel trou un trou que jaurai...,2863,2869,mais pas nimporte quel trou un trou que jaurai...,u41ujNodvnM,https://www.youtube.com/watch?v=u41ujNodvnM&t=...
2,je viens de trouver un serpent mort dans le ja...,895,902,je viens de trouver un serpent mort dans le ja...,ikuWuGn9oFM,https://www.youtube.com/watch?v=ikuWuGn9oFM&t=...
3,ce qui est important pour nos clients cest de ...,95,102,ce qui est important pour nos clients cest de ...,TplHM954vAs,https://www.youtube.com/watch?v=TplHM954vAs&t=95s
4,aussi bien dans ce qui est général que dans ce...,2927,2934,aussi bien dans ce qui est général que dans ce...,hRF_qePrIUk,https://www.youtube.com/watch?v=hRF_qePrIUk&t=...
...,...,...,...,...,...,...
195,partager aussi également une vidéo et puis vot...,253,261,partager aussi également une vidéo et puis vot...,DFze2k7f7sE,https://www.youtube.com/watch?v=DFze2k7f7sE&t=...
196,voie la annoncer la bonne nouvelle si vous vou...,592,599,voie la annoncer la bonne nouvelle si vous vou...,SvKi9oM2IVE,https://www.youtube.com/watch?v=SvKi9oM2IVE&t=...
197,quand il se retrouve pas avec au final un prod...,1244,1249,quand il se retrouve pas avec au final un prod...,zCpKkmophyg,https://www.youtube.com/watch?v=zCpKkmophyg&t=...
198,effectivement pour la chine cette grande quest...,3270,3275,effectivement pour la chine cette grande quest...,5tOuYW7k9LM,https://www.youtube.com/watch?v=5tOuYW7k9LM&t=...


In [13]:
df_10000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_10000_Word{file_ext}.csv")
df_10000 = df_10000.head(200)
df_10000

,search_string,start_time,end_time,sentence,video_id,video_url
0,même si le meilleur moyen de réduire nos déche...,345,354,même si le meilleur moyen de réduire nos déche...,eXE0dzAXErg,https://www.youtube.com/watch?v=eXE0dzAXErg&t=...
1,bonjour et bienvenue dans cette nouvelle vidéo...,30,38,bonjour et bienvenue dans cette nouvelle vidéo...,u0YcBe6UKGQ,https://www.youtube.com/watch?v=u0YcBe6UKGQ&t=30s
2,ce qui est important pour nos clients cest de ...,95,103,ce qui est important pour nos clients cest de ...,TplHM954vAs,https://www.youtube.com/watch?v=TplHM954vAs&t=95s
3,donc ma question voilà quelle est la relation ...,7261,7273,donc ma question voilà quelle est la relation ...,tlPvaDllxiI,https://www.youtube.com/watch?v=tlPvaDllxiI&t=...
4,et au bout dun moment comme je cherchais un en...,892,897,et au bout dun moment comme je cherchais un en...,GUQKIROlp2w,https://www.youtube.com/watch?v=GUQKIROlp2w&t=...
...,...,...,...,...,...,...
195,alors cest simple il suffit daller acheter du ...,7,10,alors cest simple il suffit daller acheter du ...,cCcmmDJSbko,https://www.youtube.com/watch?v=cCcmmDJSbko&t=7s
196,aller voir et sinon si ces faits directement c...,549,555,aller voir et sinon si ces faits directement c...,MEoLOIpk9aY,https://www.youtube.com/watch?v=MEoLOIpk9aY&t=...
197,en france selon moment où je sortirai cette vi...,974,981,en france selon moment où je sortirai cette vi...,qqKVtNoAB4w,https://www.youtube.com/watch?v=qqKVtNoAB4w&t=...
198,promet dêtre vraiment très intéressant bonne f...,2900,2908,promet dêtre vraiment très intéressant bonne f...,EvCDk57iWxk,https://www.youtube.com/watch?v=EvCDk57iWxk&t=...


In [14]:
df_20000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_20000_Word{file_ext}.csv")
df_20000 = df_20000.head(200)
df_20000

,search_string,start_time,end_time,sentence,video_id,video_url
0,même si le meilleur moyen de réduire nos déche...,345,354,même si le meilleur moyen de réduire nos déche...,eXE0dzAXErg,https://www.youtube.com/watch?v=eXE0dzAXErg&t=...
1,bonjour et bienvenue dans cette nouvelle vidéo...,30,38,bonjour et bienvenue dans cette nouvelle vidéo...,u0YcBe6UKGQ,https://www.youtube.com/watch?v=u0YcBe6UKGQ&t=30s
2,on vous montrera également les différentes tec...,85,94,on vous montrera également les différentes tec...,Dat9ftsSmhQ,https://www.youtube.com/watch?v=Dat9ftsSmhQ&t=85s
3,et le but clairement de cette formation cest d...,56,65,et le but clairement de cette formation cest d...,AnR_NjH8SHI,https://www.youtube.com/watch?v=AnR_NjH8SHI&t=56s
4,dailleurs petite info qui je pense pourrait vo...,68,76,dailleurs petite info qui je pense pourrait vo...,HKFpO8kinlk,https://www.youtube.com/watch?v=HKFpO8kinlk&t=68s
...,...,...,...,...,...,...
195,cest choisir les meilleures personnes pour rej...,33,40,cest choisir les meilleures personnes pour rej...,zt7oAUEdHoY,https://www.youtube.com/watch?v=zt7oAUEdHoY&t=33s
196,mais enfin estce que cest une grande valeur si...,184,194,mais enfin estce que cest une grande valeur si...,ERUED5OjyWA,https://www.youtube.com/watch?v=ERUED5OjyWA&t=...
197,et puis je suis partie avec elle chez les pers...,53,60,et puis je suis partie avec elle chez les pers...,Givk9Q88Lgo,https://www.youtube.com/watch?v=Givk9Q88Lgo&t=53s
198,du coup jai pris tous les trucs pas marrant qu...,3988,3992,du coup jai pris tous les trucs pas marrant qu...,u41ujNodvnM,https://www.youtube.com/watch?v=u41ujNodvnM&t=...


In [15]:
df_40000 = pd.read_csv(f"{parent_folder_path}/{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_40000_Word{file_ext}.csv")
df_40000 = df_40000.head(200)
df_40000

,search_string,start_time,end_time,sentence,video_id,video_url
0,mais comme toujours ce matériau nest pas parfa...,365,377,mais comme toujours ce matériau nest pas parfa...,e2ITAibdtWo,https://www.youtube.com/watch?v=e2ITAibdtWo&t=...
1,même si le meilleur moyen de réduire nos déche...,345,354,même si le meilleur moyen de réduire nos déche...,eXE0dzAXErg,https://www.youtube.com/watch?v=eXE0dzAXErg&t=...
2,chassés pour leur fourrure et leurs os utilisé...,529,539,chassés pour leur fourrure et leurs os utilisé...,ykcYguOUSSM,https://www.youtube.com/watch?v=ykcYguOUSSM&t=...
3,bonjour et bienvenue dans cette nouvelle vidéo...,30,38,bonjour et bienvenue dans cette nouvelle vidéo...,u0YcBe6UKGQ,https://www.youtube.com/watch?v=u0YcBe6UKGQ&t=30s
4,donc là tous les gens se réunissent autour de ...,1574,1584,donc là tous les gens se réunissent autour de ...,GUQKIROlp2w,https://www.youtube.com/watch?v=GUQKIROlp2w&t=...
...,...,...,...,...,...,...
195,estce que ça va être ce que jappelle souvent j...,1734,1738,estce que ça va être ce que jappelle souvent j...,Tndgvn3FoZU,https://www.youtube.com/watch?v=Tndgvn3FoZU&t=...
196,cest que les stagiaires pourraient mentionner ...,2156,2164,cest que les stagiaires pourraient mentionner ...,6TgbuYkiEIs,https://www.youtube.com/watch?v=6TgbuYkiEIs&t=...
197,tous les jours il rentrait chez lui et pouvait...,4997,5002,tous les jours il rentrait chez lui et pouvait...,fvz2uOAVdzU,https://www.youtube.com/watch?v=fvz2uOAVdzU&t=...
198,morgane ils ont mis du béton partout pour les ...,350,357,morgane ils ont mis du béton partout pour les ...,77BMeRZ9tZA,https://www.youtube.com/watch?v=77BMeRZ9tZA&t=...


#### Sending Data To Firestore

##### Language Data

In [16]:
language_data = {"languages":[{"id":1, "name":"English"},{"id":2, "name":"Turkish"},{"id":3, "name":"French"},
                              {"id":4,"name":"Spanish"},{"id":5, "name":"Portuguese"},{"id":6, "name":"Dutch"},
                              {"id":7, "name":"Italian"},{"id":8, "name":"German"},{"id":9, "name":"Arabic"}]}

In [17]:
#fire_db.collection("language-data").document("languages").set(language_data)

##### Language Detail

##### Absolute Beginner

In [16]:
# Absolute Beginner Data
end_time_list = df_28["end_time"].to_list()
video_id_list = df_28["video_id"].to_list()
start_time_list = df_28["start_time"].to_list()

In [17]:
firestore_beginner_data = zip(end_time_list,video_id_list,start_time_list)
beginner_description_text = "description"

In [18]:
beginner_data_list = []
for endtime, videoid, starttime in firestore_beginner_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    beginner_data_list.append(data)
    
#beginner_data_list   

In [19]:
data_beginner = {"description":f"{beginner_description_text}", "level":0, "levelName":"Absolute Beginner", "videos":beginner_data_list}
#data_beginner

In [20]:
#fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_beginner)

##### A1 Level

In [21]:
# A1 Level Data
end_time_list = df_200["end_time"].to_list()
video_id_list = df_200["video_id"].to_list()
start_time_list = df_200["start_time"].to_list()

In [22]:
firestore_A1_data = zip(end_time_list,video_id_list,start_time_list)
A1_description_text = "A1 level description"

In [23]:
A1_data_list = []
for endtime, videoid, starttime in firestore_A1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A1_data_list.append(data)

#A1_data_list   

In [24]:
data_A1 = {"description":f"{A1_description_text}", "level":1, "levelName":"A1", "videos":A1_data_list}
#data_A1

##### A2 Level

In [25]:
# A2 Level Data
end_time_list = df_1000["end_time"].to_list()
video_id_list = df_1000["video_id"].to_list()
start_time_list = df_1000["start_time"].to_list()

In [26]:
firestore_A2_data = zip(end_time_list,video_id_list,start_time_list)
A2_description_text = "A2 level description"

In [27]:
A2_data_list = []
for endtime, videoid, starttime in firestore_A2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    A2_data_list.append(data)

#A2_data_list   

In [28]:
data_A2 = {"description":f"{A2_description_text}", "level":2, "levelName":"A2", "videos":A2_data_list}
#data_A2

##### B1 Level

In [29]:
# B1 Level Data
end_time_list = df_5000["end_time"].to_list()
video_id_list = df_5000["video_id"].to_list()
start_time_list = df_5000["start_time"].to_list()

In [30]:
firestore_B1_data = zip(end_time_list,video_id_list,start_time_list)
B1_description_text = "B1 level description"

In [31]:
B1_data_list = []
for endtime, videoid, starttime in firestore_B1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B1_data_list.append(data)

#B1_data_list   

In [32]:
data_B1 = {"description":f"{B1_description_text}", "level":3, "levelName":"B1", "videos":B1_data_list}
#data_B1

##### B2 Level

In [33]:
# B2 Level Data
end_time_list = df_10000["end_time"].to_list()
video_id_list = df_10000["video_id"].to_list()
start_time_list = df_10000["start_time"].to_list()

In [34]:
firestore_B2_data = zip(end_time_list,video_id_list,start_time_list)
B2_description_text = "B2 level description"

In [35]:
B2_data_list = []
for endtime, videoid, starttime in firestore_B2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    B2_data_list.append(data)

#B2_data_list   

In [36]:
data_B2 = {"description":f"{B2_description_text}", "level":4, "levelName":"B2", "videos":B2_data_list}
#data_B2

##### C1 Level

In [37]:
# C1 Level Data
video_id_list = df_20000["video_id"].to_list()
end_time_list = df_20000["end_time"].to_list()
start_time_list = df_20000["start_time"].to_list()

In [38]:
firestore_C1_data = zip(end_time_list,video_id_list,start_time_list)
C1_description_text = "C1 level description"

In [39]:
C1_data_list = []
for endtime, videoid, starttime in firestore_C1_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C1_data_list.append(data)

#C1_data_list   

In [40]:
data_C1 = {"description":f"{C1_description_text}", "level":5, "levelName":"C1", "videos":C1_data_list}
#data_C1

##### C2 Level

In [41]:
# C2 Level Data
video_id_list = df_40000["video_id"].to_list()
end_time_list = df_40000["end_time"].to_list()
start_time_list = df_40000["start_time"].to_list()

In [42]:
firestore_C2_data = zip(end_time_list,video_id_list,start_time_list)
C2_description_text = "C2 level description"

In [43]:
C2_data_list = []
for endtime, videoid, starttime in firestore_C2_data:
    data = {"endTime":endtime,
            "id":videoid,
            "startTime":starttime}
    C2_data_list.append(data)

#C2_data_list   

In [44]:
data_C2 = {"description":f"{C2_description_text}", "level":6, "levelName":"C2", "videos":C2_data_list}
#data_C2

##### All Data Sending

In [45]:
data_all = {"datas":[data_beginner, data_A1, data_A2, data_B1, data_B2, data_C1, data_C2]}
#data_all

In [46]:
fire_db.collection("language-detail-data").document(f"{lang_id}").set(data_all)

update_time {
  seconds: 1671709565
  nanos: 893572000
}

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass